# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [2]:
!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [3]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [4]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [6]:
# env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

In [7]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [9]:
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
import time

from ddpg_agent import Agent

In [10]:
env_info = env.reset(train_mode=True)[brain_name]
# env.seed(10)
states = env_info.vector_observations
state_size = states.shape[1]
action_size = brain.vector_action_space_size

agent = Agent(state_size=state_size, action_size=action_size, random_seed=10)

In [ ]:
def ddpg(n_episodes=2000, max_t=1000, goal_score=30, learn_every=20, num_learn=10):
    total_score_deque = deque(maxlen=100)
    total_score = []
    max_score = -np.Inf
    run_times = []
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
        states = env_info.vector_observations                 # get the current state (for each agent)
        agent.reset()
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        step_counter = 0
        run_times.append(time.time())
        while True:
            step_counter += 1
            actions = agent.act(states)
            env_info = env.step(actions)[brain_name]           # send all actions to tne environment
            next_states = env_info.vector_observations         # get next state (for each agent)
            rewards = env_info.rewards                        # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            scores += env_info.rewards                         # update the score (for each agent)

            agent.step(states, actions, rewards, next_states, dones)

            states = next_states                              # roll over states to next time step

            if np.any(dones):
                break
        run_times[-1] = time.time() - run_times[-1]
        mean_score = np.mean(scores)
        total_score_deque.append(mean_score)
        total_score.append(mean_score)
        total_average_score = np.mean(total_score_deque)
        total_run_time = np.sum(run_times)
        average_run_time = np.mean(run_times)

        print('\rEpisode {}\tLast {} Average Score: {:.2f}\tAverage Score: {:.2f}\tNum steps {}\tTime {}'.format(
            i_episode, len(total_score_deque), total_average_score, mean_score, step_counter, average_run_time, end=""))
        if max_score < total_average_score:
            max_score = total_average_score
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            print('\rNew Hi-Score! Episode {}\tLast {} Average Score: {:.2f}'.format(
                i_episode, len(total_score_deque), total_average_score))
        if total_average_score >= goal_score and len(total_score_deque) == total_score_deque.maxlen:
            print('\rGoal achieved at episode {} with score {}'.format(i_episode, total_average_score))
            print('\rTotal run time {}\t Average episode time {}'.format(total_run_time, average_run_time, end=''))
        
    return total_score

total_score = ddpg()

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(1, len(total_score)+1), total_score)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()


Episode 1	Last 1 Average Score: 0.72	Average Score: 0.72	Num steps 1001	Time 17.450274229049683
New Hi-Score! Episode 1	Last 1 Average Score: 0.72
Episode 2	Last 2 Average Score: 1.01	Average Score: 1.29	Num steps 1001	Time 18.060277581214905
New Hi-Score! Episode 2	Last 2 Average Score: 1.01
Episode 3	Last 3 Average Score: 1.28	Average Score: 1.84	Num steps 1001	Time 18.04983417193095
New Hi-Score! Episode 3	Last 3 Average Score: 1.28
Episode 4	Last 4 Average Score: 1.75	Average Score: 3.15	Num steps 1001	Time 18.081406950950623
New Hi-Score! Episode 4	Last 4 Average Score: 1.75
Episode 5	Last 5 Average Score: 2.09	Average Score: 3.45	Num steps 1001	Time 18.17175335884094
New Hi-Score! Episode 5	Last 5 Average Score: 2.09
Episode 6	Last 6 Average Score: 2.46	Average Score: 4.33	Num steps 1001	Time 18.275147557258606
New Hi-Score! Episode 6	Last 6 Average Score: 2.46
Episode 7	Last 7 Average Score: 2.77	Average Score: 4.60	Num steps 1001	Time 18.31862732342311
New Hi-Score! Episode 7	L